In [1]:
# Imports using Sklearn make shortcut functions
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from datetime import datetime, timedelta

In [2]:
# Print current time
now = datetime.now()
current_time = now.strftime("%Y-%m-%d %H:%M:%S")

In [3]:
# Imports data
data = pd.read_csv('../processed_data/1988_to_2024_combined_clean_polling_and_results.csv')

In [4]:
data.rename(columns={'OTH_PERCENTAGE': 'OTH_ACT'}, inplace=True)

In [5]:
# Convert date columns to datetime
data['enddate'] = pd.to_datetime(data['enddate'])
data['next_elec_date'] = pd.to_datetime(data['next_elec_date'])
data['startdate'] = pd.to_datetime(data['startdate'])

In [6]:
# Handle numerical transformer
num_columns_selector = ['samplesize', 'months_to_elec_weight']
num_transformer = MinMaxScaler()

In [7]:
# Handle categorical transformer
cat_columns_selector = ['rating']
cat_transformer = make_pipeline(OrdinalEncoder(categories = [['F','F+','E-','E','E+','D-','D','D+','C-','C','C+','B-','B','B+','A-']]),MinMaxScaler())

In [8]:
# One-hot encoder for party_in_power
ohe = OneHotEncoder()

In [9]:
# Build the preprocessing pipeline
preproc_pipeline = make_column_transformer(
    (num_transformer, num_columns_selector),
    (cat_transformer, cat_columns_selector),
    (ohe,['party_in_power']),
    remainder='passthrough',
    verbose_feature_names_out=False
)

In [10]:
data['next_elec_date'].value_counts()

next_elec_date
2015-05-07    1930
2024-07-04    1362
2010-05-06     584
1992-04-09     453
2019-12-12     440
1997-05-01     338
2017-06-08     254
2001-06-07     213
2005-05-05      98
Name: count, dtype: int64

In [13]:
# Define election dates
election_date = datetime.strptime('2024-07-04', '%Y-%m-%d')
cutoff_date = election_date - timedelta(days=84)
prediction_date = election_date - timedelta(days=30)

In [14]:
# Split data
data_train = data[data['startdate'] < cutoff_date]
data_test_1 = data[(data['startdate'] >= cutoff_date) & (data['startdate'] < prediction_date)]
data_test = data_test_1[data_test_1['next_elec_date'] == election_date]


In [15]:
# Fit transform preprocessing pipeline to data_train
data_train_processed = preproc_pipeline.fit_transform(data_train)


In [16]:
# Transform preprocessing pipeline to data_test
data_test_processed = preproc_pipeline.transform(data_test)

In [17]:
# Check feature names make sense for data_train
data_train_processed = pd.DataFrame(
    data_train_processed, columns=preproc_pipeline.get_feature_names_out()
)

# Check feature names make sense for data_test
data_test_processed = pd.DataFrame(
    data_test_processed, columns=preproc_pipeline.get_feature_names_out()
)

In [18]:
# Define our X by dropping irrelevant and y columns

X_train = data_train_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])
X_test = data_test_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])

In [19]:
# Build our target matrix
y_train = data_train_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]
y_test = data_test_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]

In [20]:
# Drops y_train actuals where the actual is the actual result we are trying to predict, replaces with NaNs
y_train.loc[y_train['next_elec_date'] == '2024-07-04',
         ['LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']] = np.nan

In [21]:
y_train

,next_elec_date,LAB_ACT,CON_ACT,LIB_ACT,GRE_ACT,BRX_ACT,SNP_ACT,UKI_ACT,PLC_ACT,OTH_ACT
0,1992-04-09,0.343843,0.419323,0.178082,0.004797,0.0,0.018731,0.0,0.004664,0.030561
1,1992-04-09,0.343843,0.419323,0.178082,0.004797,0.0,0.018731,0.0,0.004664,0.030561
2,1992-04-09,0.343843,0.419323,0.178082,0.004797,0.0,0.018731,0.0,0.004664,0.030561
3,1992-04-09,0.343843,0.419323,0.178082,0.004797,0.0,0.018731,0.0,0.004664,0.030561
4,1992-04-09,0.343843,0.419323,0.178082,0.004797,0.0,0.018731,0.0,0.004664,0.030561
...,...,...,...,...,...,...,...,...,...,...
5611,2024-07-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5612,2024-07-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5613,2024-07-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5614,2024-07-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Calculates average median value of X_test
averages = X_test.mean()
averages

samplesize                      0.01376
months_to_elec_weight          0.706512
rating                         0.640306
party_in_power_Conservative         1.0
poll_length                    1.928571
CON_FC                         0.233571
LAB_FC                         0.441071
LIB_FC                         0.095536
BRX_FC                         0.116071
GRE_FC                         0.059286
OTH_FC                         0.022727
PLC_FC                           0.0085
SNP_FC                         0.026429
UKI_FC                              NaN
dtype: object

In [29]:
# Creates imputation values for y_train to impute over actuals we are trying to predict
imputation_values = {
    'CON_ACT': averages['CON_FC'],
    'LAB_ACT': averages['LAB_FC'],
    'LIB_ACT': averages['LIB_FC'],
    'BRX_ACT': averages['BRX_FC'],
    'GRE_ACT': averages['GRE_FC'],
    'OTH_ACT': averages['OTH_FC'],
    'PLC_ACT': averages['PLC_FC'],
    'SNP_ACT': averages['SNP_FC'],
    'UKI_ACT': 0
}

In [30]:
# Applies imputation values to y_train
y_train = y_train.fillna(value=imputation_values)

In [31]:
# Instantiate the model for each party
models = {}
parties = ['CON', 'LAB', 'LIB', 'BRX', 'GRE', 'SNP', 'UKI', 'PLC', 'OTH']
for party in parties:
    models[party] = XGBRegressor(
        learning_rate=0.3, n_estimators=300, max_depth=3, subsample=0.7,
        objective='reg:squarederror', nthread=-1, enable_categorical=True
    )

In [32]:
# Train models
X_train_matrix = np.array(X_train)
for party in parties:
    models[party].fit(X_train_matrix, y_train[f'{party}_ACT'])

In [33]:
# Evaluate model performance with cross validation
cv_scores = {}
for party in parties:
    cv_scores[party] = cross_val_score(models[party], X_train_matrix, y_train[f'{party}_ACT']).mean()

In [34]:
# Evaluate predictions
X_test_matrix = np.array(X_test)
mean_predictions = {}
for party in parties:
    mean_predictions[party] = models[party].predict(X_test_matrix).mean()


In [35]:
# Print mean predictions
for party in parties:
    print(f"2024 mean prediction for {party}: {mean_predictions[party]}")


2024 mean prediction for CON: 0.24076144397258759
2024 mean prediction for LAB: 0.43921345472335815
2024 mean prediction for LIB: 0.09615784138441086
2024 mean prediction for BRX: 0.10929613560438156
2024 mean prediction for GRE: 0.056187860667705536
2024 mean prediction for SNP: 0.032949693500995636
2024 mean prediction for UKI: 0.00033923727460205555
2024 mean prediction for PLC: 0.007681995630264282
2024 mean prediction for OTH: 0.029095053672790527


In [36]:
# Evaluate last five polls
last_predictions = {}
for party in parties:
    last_predictions[party] = models[party].predict(X_test_matrix)[-5:]

In [38]:
# Print mean of predictions
for party in parties:
    print(f"2024 last prediction for {party}: {last_predictions[party].mean()}")


2024 last prediction for CON: 0.24293765425682068
2024 last prediction for LAB: 0.43703216314315796
2024 last prediction for LIB: 0.09661276638507843
2024 last prediction for BRX: 0.1091739684343338
2024 last prediction for GRE: 0.05613730475306511
2024 last prediction for SNP: 0.0366714708507061
2024 last prediction for UKI: 0.0006372841307893395
2024 last prediction for PLC: 0.0075058043003082275
2024 last prediction for OTH: 0.03256234899163246
